# Anthropic Client - Complex Multi-Tool Test

This notebook tests the `AnthropicAgent` with:
- File uploads (Files API)
- Multiple tools: code execution, web search, web fetch, and custom tools
- Multi-turn conversation with tool use
- Streaming with retry mechanism


In [1]:
import asyncio
import anthropic
from src.core.agent import AnthropicAgent
from dotenv import load_dotenv

# Load environment variables
load_dotenv()


True

## Step 1: Upload a File (Optional)

Upload a PDF file using the Anthropic Files API. Replace the file path with your own PDF.


In [2]:
# Initialize Anthropic client for file upload
client = anthropic.Anthropic()

# Upload a file (uncomment and modify path as needed)
# with open('path/to/your/file.pdf', 'rb') as file:
#     uploaded_file = client.beta.files.upload(
#         file=('your-file.pdf', file, 'application/pdf')
#     )
#     file_id = uploaded_file.id
#     print(f"Uploaded file: {uploaded_file}")

# For testing without file upload, use a placeholder
file_id = "file_011CV2R3aJGA4a655cY3DEYP"  # Set to your file_id if you uploaded a file
# file_id = None
print(f"File ID: {file_id}")


File ID: file_011CV2R3aJGA4a655cY3DEYP


In [5]:
files = client.beta.files.list()
files.data

[FileMetadata(id='file_011CV58uEKQBMS9xfJvzZEcM', created_at=datetime.datetime(2025, 11, 13, 2, 56, 47, 873000, tzinfo=datetime.timezone.utc), filename='tata-motor-first-page.png', mime_type='image/png', size_bytes=2068787, type='file', downloadable=True),
 FileMetadata(id='file_011CV57gGc5YqTc5vx6tsG44', created_at=datetime.datetime(2025, 11, 13, 2, 40, 45, 230000, tzinfo=datetime.timezone.utc), filename='tata_motor_first_page.png', mime_type='image/png', size_bytes=4080682, type='file', downloadable=True),
 FileMetadata(id='file_011CV57PbQhns6TnWq3tXs3D', created_at=datetime.datetime(2025, 11, 13, 2, 36, 59, 8000, tzinfo=datetime.timezone.utc), filename='tata_motor_page1.png', mime_type='image/png', size_bytes=2068787, type='file', downloadable=True),
 FileMetadata(id='file_011CV516McZpMSQ548Ld5wYs', created_at=datetime.datetime(2025, 11, 13, 1, 14, 24, 232000, tzinfo=datetime.timezone.utc), filename='first_page.png', mime_type='image/png', size_bytes=2068787, type='file', downloadab

In [7]:
for file in files.data:
    client.beta.files.delete(file.id)

## Step 2: Define Tools

Define the tools that the agent will use:
1. **Code Execution**: Server-side code execution (beta feature)
2. **Web Search**: Server-side web search (beta feature)
3. **Web Fetch**: Server-side web fetching (beta feature)
4. **get_weather**: Custom client-side tool (requires manual execution)


In [3]:
# Define custom get_weather tool
get_weather_tool = {
    "name": "get_weather",
    "description": "Get the current weather in a given location",
    "input_schema": {
        "type": "object",
        "properties": {
            "location": {
                "type": "string",
                "description": "The city and state, e.g. San Francisco, CA"
            },
            "unit": {
                "type": "string",
                "enum": ["celsius", "fahrenheit"],
                "description": "The unit of temperature, either 'celsius' or 'fahrenheit'"
            }
        },
        "required": ["location"]
    }
}

# Define all tools (server-side + custom)
tools = [
    {
        "type": "code_execution_20250825",
        "name": "code_execution"
    },
    {
        "type": "web_search_20250305",
        "name": "web_search",
        "max_uses": 10
    },
    {
        "type": "web_fetch_20250910",
        "name": "web_fetch",
        "max_uses": 10
    },
    get_weather_tool
]

# Beta features
beta_headers = [
    "files-api-2025-04-14",
    "code-execution-2025-08-25",
    "web-fetch-2025-09-10"
]

print(f"Configured {len(tools)} tools")
print(f"Beta features: {beta_headers}")


Configured 4 tools
Beta features: ['files-api-2025-04-14', 'code-execution-2025-08-25', 'web-fetch-2025-09-10']


## Step 3: Define Tool Executor

The tool executor handles client-side tool calls (like `get_weather`). Server-side tools (code execution, web search) are executed automatically by Anthropic.


In [4]:
def execute_tool(tool_name: str, tool_input: dict) -> str:
    """Execute client-side tools.
    
    Server-side tools (code_execution, web_search, web_fetch) are handled
    automatically by Anthropic and don't need execution here.
    """
    if tool_name == "get_weather":
        location = tool_input.get("location", "Unknown")
        unit = tool_input.get("unit", "fahrenheit")
        
        # Simulate weather API call (replace with real API in production)
        temp = 60 if unit == "fahrenheit" else 15
        condition = "cloudy"
        
        return f"The current weather in {location} is {temp}°{unit[0].upper()} and {condition}."
    
    else:
        return f"Unknown tool: {tool_name}"

# Test the tool executor
test_result = execute_tool("get_weather", {"location": "San Francisco, CA", "unit": "fahrenheit"})
print(f"Tool test: {test_result}")


Tool test: The current weather in San Francisco, CA is 60°F and cloudy.


## Step 4: First Turn - Complex Multi-Tool Request

Send a complex request that may use multiple tools:
- If `file_id` is set: Extract first page of PDF as image (requires code execution)
- Get weather in San Francisco (requires custom get_weather tool)


In [9]:
async def run_complex_agent_test():
    """Run a complex multi-tool agent test."""
    
    # Build the message content
    if file_id:
        message_content = [
            {"type": "container_upload", "file_id": file_id},
            {"type": "text", "text": "Extract the first page of the pdf as an image and return it to me. Also get the weather in San Francisco."}
        ]
    else:
        # Simplified request without file upload
        message_content = "Get the weather in San Francisco and tell me a fun fact about the city."
    
    # Initialize the agent
    agent = AnthropicAgent(
        system_prompt="Solve user's problem step by step. Be concise and helpful.",
        model="claude-sonnet-4-5",
        max_steps=10,
        thinking_tokens=1024,
        max_tokens=16000,
        tools=tools,
        beta_headers=beta_headers,
        max_retries=3,
        base_delay=1.0
    )
    
    # Create queue for streaming
    queue = asyncio.Queue()
    
    # Sentinel value to signal completion
    DONE = object()
    
    # Stream consumer with proper coordination
    async def consume_stream():
        """Consume and display streaming output."""
        while True:
            chunk = await queue.get()
            if chunk is DONE:
                break
            print(chunk, end="", flush=True)
    
    # Wrapper to add sentinel after agent completes
    async def run_agent_with_sentinel():
        """Run agent and signal completion."""
        try:
            result = await agent.run(
                prompt=message_content,
                queue=queue,
                tool_executor=execute_tool
            )
            return result
        finally:
            # Always signal completion, even on error
            await queue.put(DONE)
    
    # Run the agent
    print("=" * 80)
    print("FIRST TURN - Complex Multi-Tool Request")
    print("=" * 80)
    print(f"\nUser: {message_content if isinstance(message_content, str) else message_content[-1]['text']}\n")
    print("Agent Response:")
    print("-" * 80)
    
    # Start both tasks concurrently
    agent_task = asyncio.create_task(run_agent_with_sentinel())
    stream_task = asyncio.create_task(consume_stream())
    
    # Wait for completion
    try:
        result = await agent_task
        await stream_task
        
        print("\n" + "-" * 80)
        print(f"\n✓ First turn completed successfully")
        print(f"  Stop reason: {result.stop_reason}")
        print(f"  Container ID: {agent.container_id}")
        print(f"  Total messages: {len(agent.messages)}")
        
        return agent, result
        
    except Exception as e:
        print(f"\n✗ Error: {e}")
        stream_task.cancel()
        raise

# Run the first turn
agent, result1 = await run_complex_agent_test()

FIRST TURN - Complex Multi-Tool Request

User: Extract the first page of the pdf as an image and return it to me. Also get the weather in San Francisco.

Agent Response:
--------------------------------------------------------------------------------
<nova-labs:thinking>The user wants me to:
1. Extract the first page of the PDF as an image
2. Get the weather in San Francisco

Let me start by getting the weather in San Francisco and examining the PDF file to extract the first page as an image.

First, I'll need to check the uploaded PDF file and then convert the first page to an image. I'll also get the weather for San Francisco.

Let me call both tools - the weather tool and bash to work with the PDF.
</nova-labs:thinking><nova-labs:text>I'll help you extract the first page of the PDF as an image and get the weather in San Francisco.
</nova-labs:text><nova-labs:tool_call>
{
  "type": "tool_use",
  "id": "toolu_01Nyz5PKFNHYDaT9cXtybPGx",
  "name": "get_weather",
  "input": {
    "locati

## Step 5: Second Turn - Follow-up Question

Continue the conversation with a follow-up question. The agent maintains the conversation history and container state.


In [10]:
async def run_second_turn():
    """Run a second turn with follow-up question."""
    
    # Follow-up question
    follow_up = "What did you do in the previous step? Summarize briefly."
    
    # Create queue for streaming
    queue = asyncio.Queue()
    
    # Stream consumer
    async def consume_stream():
        """Consume and display streaming output."""
        while True:
            try:
                chunk = await asyncio.wait_for(queue.get(), timeout=0.1)
                print(chunk, end="", flush=True)
            except asyncio.TimeoutError:
                if agent_task.done():
                    # Drain remaining
                    while not queue.empty():
                        chunk = queue.get_nowait()
                        print(chunk, end="", flush=True)
                    break
            except Exception as e:
                print(f"\nError consuming stream: {e}")
                break
    
    # Run the agent
    print("\n" + "=" * 80)
    print("SECOND TURN - Follow-up Question")
    print("=" * 80)
    print(f"\nUser: {follow_up}\n")
    print("Agent Response:")
    print("-" * 80)
    
    agent_task = asyncio.create_task(
        agent.run(
            prompt=follow_up,
            queue=queue,
            tool_executor=execute_tool
        )
    )
    
    stream_task = asyncio.create_task(consume_stream())
    
    # Wait for completion
    try:
        result = await agent_task
        await stream_task
        
        print("\n" + "-" * 80)
        print(f"\n✓ Second turn completed successfully")
        print(f"  Stop reason: {result.stop_reason}")
        print(f"  Total messages: {len(agent.messages)}")
        
        return result
        
    except Exception as e:
        print(f"\n✗ Error: {e}")
        stream_task.cancel()
        raise

# Run the second turn
result2 = await run_second_turn()



SECOND TURN - Follow-up Question

User: What did you do in the previous step? Summarize briefly.

Agent Response:
--------------------------------------------------------------------------------
<nova-labs:thinking>The user is asking me to summarize what I did in the previous step. Let me briefly explain the actions I took:

1. Got the weather in San Francisco
2. Located the uploaded PDF file
3. Used Python with pdf2image library to convert the first page of the PDF to a PNG image
4. Saved the image to the OUTPUT_DIR so it could be exported to the user

I should keep this concise and clear.
</nova-labs:thinking><nova-labs:text>In the previous step, I:

1. **Retrieved the weather** for San Francisco (60°F and cloudy)
2. **Located your PDF file** in the input directory
3. **Converted the first page** of the PDF to a PNG image using Python's pdf2image library
4. **Exported the image** (tata-motor-first-page.png) for you to download

Essentially, I extracted the first page of your Tata Mo

In [11]:
result1

BetaMessage(id='msg_01BDcFGQ5UEV9QDjmgnZLkgS', container=BetaContainer(id='container_011CV58sgf7Tp9uTP3DCQ8zZ', expires_at=datetime.datetime(2025, 11, 13, 3, 56, 48, 264662, tzinfo=TzInfo(UTC)), skills=None), content=[BetaBashCodeExecutionToolResultBlock(content=BetaBashCodeExecutionResultBlock(content=[], return_code=2, stderr="ls: cannot access '/files/input/1e8708632b2c1208': No such file or directory\n", stdout='', type='bash_code_execution_result'), tool_use_id='srvtoolu_015iuAvt7SM7HvKBNgNVnn4j', type='bash_code_execution_tool_result'), BetaServerToolUseBlock(id='srvtoolu_017j73wnEnLXL55RtCNeHRmi', input={'command': 'find /files -name "*.pdf" 2>/dev/null'}, name='bash_code_execution', type='server_tool_use'), BetaBashCodeExecutionToolResultBlock(content=BetaBashCodeExecutionResultBlock(content=[], return_code=0, stderr='', stdout='/files/input/1e8708632b2c1208/tata-motor-IAR-2024-25.pdf\n', type='bash_code_execution_result'), tool_use_id='srvtoolu_017j73wnEnLXL55RtCNeHRmi', type=

## Step 6: Conversation Summary

Review the complete conversation history and inspect the message structure.


In [ ]:
import json

print("=" * 80)
print("CONVERSATION SUMMARY")
print("=" * 80)

print(f"\nTotal Messages: {len(agent.messages)}")
print(f"Container ID: {agent.container_id}")
print(f"Model: {agent.model}")
print(f"Max Steps: {agent.max_steps}")

print("\n" + "-" * 80)
print("Message History:")
print("-" * 80)

for i, msg in enumerate(agent.messages, 1):
    role = msg.get('role', 'unknown')
    content = msg.get('content', [])
    
    print(f"\n{i}. [{role.upper()}]")
    
    if isinstance(content, str):
        print(f"   Content: {content[:100]}...")
    elif isinstance(content, list):
        for j, item in enumerate(content):
            if isinstance(item, dict):
                item_type = item.get('type', 'unknown')
                print(f"   [{j}] Type: {item_type}")
                
                if item_type == 'text':
                    text = item.get('text', '')
                    print(f"       Text: {text[:80]}...")
                elif item_type == 'tool_result':
                    tool_id = item.get('tool_use_id', '')
                    result = item.get('content', '')
                    print(f"       Tool ID: {tool_id}")
                    print(f"       Result: {result[:80]}...")
                elif item_type == 'container_upload':
                    print(f"       File ID: {item.get('file_id', '')}")
            elif hasattr(item, 'type'):
                print(f"   [{j}] Type: {item.type}")
                if item.type == 'tool_use':
                    print(f"       Tool: {item.name}")
                    print(f"       ID: {item.id}")
                    print(f"       Input: {str(item.input)[:80]}...")
                elif item.type == 'text':
                    print(f"       Text: {item.text[:80]}...")

print("\n" + "=" * 80)
print("✓ Test Completed Successfully!")
print("=" * 80)
